# Effect of Informal Care on Labor Supply: A Macroeconomic Perspective

## 2018.3.12 ICES, Hosei University

### Daisuke Moriwaki

Cyberagent, Inc., AI Lab        
Keio University, School of Medicine

<daisuke.moriwaki@gmail.com>

In [573]:
using QuantEcon
using Plots
using Interpolations
using Distributions

# Motivation
* Increase in the number of care recievers in Japan is inevitable. On the other hand, the human and financial resource that we can spend on care givers is getting scarse.
* It is easy to predict that "caregiver ratio" (the number of caregivers per one care receiver) explodes in the near future.

* The explosion of caregiver ratio cause several problems
    * insufficient care for elderly (insufficient availability of cheap nursing home, tokuyo)
    * Poverty due to care-leave for frail older family
    * Financial Problem for both national government and local administration
    * *Economy itself* due to less labor supply
* Such a wide range of problem should be tackled by macroeconomic model rather than causality studies.

# Literature

## labor supply and informal care

* moriwaki (2015, mimeo) finds negative effect from informal care only for male.

## care and macro economy


## heterogeneous agent and social security programs
* Li (EER, 2018)
* Kitao

# Setting

## Household

* you may consider either single households without gender and unknown re-production technique or many households with couples and kids where income, assets, health, and time endowment is shared. More realistic modelling is listed as future study
* 5 status space
    * age from 20 to 99 (5 year interval)
    * health status from 0 to 1. health with 0.1 or below becomes a care receiver.
    * wage rate is exogenous. Given wage rate, labor supply is endogeneous.
    * asset
    * *care* status; how many hours the household must spend for caring older frail family

## Preference
* simple utility function. setting is same as Yue (2018, EER)

In [574]:
function util(c::Real, l::Real,chi::Real = 0.4, sigma::Real = 0.28) # chi = 0.4
    ((c^(1-chi)*(l^chi))^(1-sigma))/(1 - sigma)
end

util (generic function with 3 methods)

## consumption
* consumption is deterimined by assets, income and saving
* there is a social assistence for the poor

## income
* income is determined by hourking hour in market and tax


## labor supply
* labor supply is endogeneous
* healthy household optimizes labor supply as well as consumption (saving) for given time endowment
* time endowment is varying across agent due to informal care
* From the first order condition $l = c(1 - \tau)w \frac{\chi}{1 - \chi}$

## Informal care
* Population is devided by health status. One is (potential) caregivers who are healthy enough to provide care to family, the other is care receivers who needs care from either/both formal or/and informal caregivers.
* Suppose most of the households prefer formal care, e.g. nursing home, to informal care at home. Then informal care occurs only when formal care provision is insufficient and this is exactly the case in Japan.
* Informal care is defined as unmet care demand from carereceivers. Informal care is distributed romdomly to caregivers according to caregivers' age.

## Government
* government provide insufficient formal care funded by social security premium contributed by working age, which is resemble to Japanese LTCI
* cost of formal care is determined by personal expenses of informal care worker

# Others
* wage ($w$) and capital cost $r$ is arbitrary determined by the nature (me).
* thus no equillibrium condition holds so far.
* $r = 0.01$, $\mathbb{E}(w) = 1$

## Quantitative Excercise
* The model is simulated with parameters set to values taken from existing literature.
* Unfortunately, I can only show the result of toy model so far.

## The toy model
* Households have only three period: young, middle, and old
* Young work only in the market and not give care.
* Middle work both in home and the market, but does not receive care.
* Old does not work.
* Wage is stochastic without persistent shocks.
* 

In [575]:
r = 0.0
tau = 0.1 # income tax
beta = 0.9 # discount rate
rep_rate = 0.5 # replacement rate
NH_rate = 0.9 # government formal care provision rate. ratio to needed care
wage_care = 0.7 # care workers' wage rate. ratio to all industry

0.5

In [1]:
# calculating utility for old given asset and benefit
# need calculation for average_wage
function value_for_old(a::Real, average_wage::Real)
    b = rep_rate*average_wage # pension benefit
    return util((a*(1 + r) + b), 1)
end

value_for_old (generic function with 1 method)

In [1]:
# caluculating value for middle
# need calculaation for average wage
function value_for_middle(a_new::Real, l::Real, a::Real, w::Real, h_H::Real, v_new::Real, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if c <= 0  || l <= 0 || 1 - h_H - l < 0
        v = -100
    else
        v = util(c, l) + beta*v_new
    end
    return v, a_new, l, a, w, h_H, y, c
end

value_for_middle (generic function with 3 methods)

In [578]:
# maximize value for middle given asset, wage and informal care
# row is new_asset x leisure
# column is wage x h_H x asset

min_asset = 0;
max_asset = 10;
min_wage = 0;
max_wage = 10;
min_h_H =0;
max_h_H = 1;
num_grid = 9; #9
min_l = 0;
max_l = 1;
asset_grid = linspace(min_asset, max_asset, num_grid);
wage_grid = linspace(min_wage, max_wage, num_grid);
h_H_grid = linspace(min_h_H, max_h_H, num_grid);
l_grid = linspace(min_l, max_l, num_grid);

In [579]:
# calculate value for each input and return maximum value for each
# warinig: no interpolation embeded!
function generate_max_values_matrix(value_function::Function, asset_grid::LinSpace = asset_grid, l_grid::LinSpace = l_grid, wage_grid::LinSpace = wage_grid, 
        h_H_grid::LinSpace = h_H_grid, num_grid::Int = num_grid)
    # initializing matrices
    value_matrix = zeros(num_grid^2, num_grid^3);
    c_mat = zeros(num_grid^2, num_grid^3);
    l_mat = zeros(num_grid^2, num_grid^3);
    y_mat = zeros(num_grid^2, num_grid^3);
    a_new_mat = ones(num_grid^2, num_grid^3);
    w_mat = zeros(num_grid^2, num_grid^3);
    a_mat = zeros(num_grid^2, num_grid^3);
    h_H_mat = zeros(num_grid^2, num_grid^3);
    for i = 1:num_grid
        for j = 1:num_grid
            for k = 1:num_grid
                for m = 1:num_grid
                    for n = 1:num_grid
                        result = value_function(asset_grid[i], l_grid[j], asset_grid[k], wage_grid[m], h_H_grid[n])
                        value_matrix[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[1]
                        a_new_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[2]
                        l_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[3]
                        a_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[4]
                        w_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[5]
                        h_H_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[6]
                        y_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[7]
                        c_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2]  = result[8]
                    end
                end
            end
        end
    end
    # get max value for each asset, wage, h_H. maximizing with respect to new asset and leisure
    #max_value = maximum(value_matrix, 1)
    max_value, max_ind = findmax(value_matrix, 1)
    # reshape value matrix to asset_grid x (w x h_H)
    # this should be fix throughout the imulation
    max_values_reshaped = reshape(max_value, num_grid, num_grid^2);
    return max_values_reshaped, max_ind, a_new_mat, l_mat, a_mat, w_mat, h_H_mat, y_mat, c_mat
end


generate_max_values_matrix (generic function with 6 methods)

In [580]:
# get policy function for a_new, h_M (Labor supply), c, 
max_values_reshaped, max_ind, a_new_mat, l_mat, a_mat, w_mat, h_H_mat, y_mat, c_mat = generate_max_values_matrix(value_for_middle);
# policy function for new asset
a_new_pol_middle = a_new_mat[max_ind]
# policy function for labor supply
l_pol_middle = l_mat[max_ind]
h_M_pol_middle = 1 - l_pol_middle - reshape(h_H_mat[1, :], 1, num_grid^3)
c_pol_middle = c_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]


In [581]:
plot(reshape(c_pol_middle, 729, 1))
#plot(hcat(reshape(w_mat[1,:], 729,1), reshape(l_pol_middle, 729, 1)))

In [582]:
# need to be removed after get_value_for_middle_with_uncertainty is modified.

"get index for a value between grid point"
function get_float_index(x::Real, grid::LinSpace, num_grid::Int = num_grid)
# get floating index from asset grid for given scalar asset level
    if x < grid[1]
        return 1
    end
    if x> grid[num_grid]       
        return num_grid
    end
    for i = 1:num_grid
        if x >= grid[i]
            return i + (x - grid[i])/(grid[i+1] - grid[i])
        end
    end
end

get_float_index

In [583]:
max_values_matrix = generate_max_values_matrix(value_for_middle)[1]

9×81 Array{Float64,2}:
 -10000.0      1.91961  2.58975  3.08553  …  -10000.0  -10000.0  -10000.0
      1.52944  2.56141  3.07029  3.49048     -10000.0  -10000.0  -10000.0
      2.90594  3.18112  3.52628  3.85687     -10000.0  -10000.0  -10000.0
      3.43987  3.70338  3.93615  4.19851     -10000.0  -10000.0  -10000.0
      3.92041  4.10833  4.31542  4.52209     -10000.0  -10000.0  -10000.0
      4.31542  4.50334  4.67092  4.84203  …  -10000.0  -10000.0  -10000.0
      4.67092  4.82866  4.99624  5.1486      -10000.0  -10000.0  -10000.0
      4.99624  5.1486   5.28903  5.4303      -10000.0  -10000.0  -10000.0
      5.28903  5.4303   5.57073  5.70151     -10000.0  -10000.0  -10000.0

In [584]:
############ need refactoring. interpolate accept "gridded" interpolation that replace get_float_index function!!!!!!!!!!!!!!!!!

# interpolation for a (new asset for young, asset in the last period for middle)
# get a = a_new and return value for middle with uncertainty
function get_value_for_middle_with_uncertainty(a_new::Real, asset_grid::LinSpace = asset_grid, max_values_matrix = max_values_matrix)
    v = zeros(num_grid^2, 1)
    a_new_ind = get_float_index(a_new, asset_grid) # get index of asset level in asset grid
    for i = 1:size(max_values_matrix, 2)
        itp = interpolate(max_values_matrix[:,i],BSpline(Linear()), OnGrid())
        v[i] = itp[a_new_ind]
    end
    return v
end

get_value_for_middle_with_uncertainty (generic function with 3 methods)

In [585]:
#plotly()
#plot(value_for_middle_mat[:, 30])
#plot(reshape(max_values, num_grid^3, 1))
#plot(max_values_reshaped)


## Distribution 
* assign probability distributions for wage and care need to each bin arbitrary
* wage follows lognormal, care need follow gamma distribution

In [586]:
# generate probability distribution for w and h_H
# wage is distributed according to Gauss


# calcurate probability at midpoint between each grid 
function get_Q(dist, grid::LinSpace, num_grid::Int=num_grid)
    Q = zeros(num_grid, 1)
    for i in 1:num_grid
        if i == 1
            Q[i] = cdf(dist, (grid[i] + grid[i + 1])/2)
        end
        if i == num_grid
            Q[i] = 1 - cdf(dist, (grid[i - 1] + grid[i])/2) 
        end
        if i > 1 && i < num_grid
            Q[i] = cdf(dist, (grid[i] + grid[i + 1])/2) - cdf(dist, (grid[i - 1] + grid[i])/2)
        end
    end
    return Q
end



get_Q (generic function with 2 methods)

In [587]:
w_dist = LogNormal(0.5, 0.5)
h_H_dist = Gamma(0.6, 0.1) # Gamma(0.5, 0.1)

# generate Q
Q_h_H = get_Q(h_H_dist, h_H_grid);
Q_w = get_Q(w_dist, wage_grid);
Q = kron(Q_w, Q_h_H);

In [588]:
# calculating value for young
function value_for_young(a_new::Real, l::Real, w::Real, Q::Array{Float64, 2}=Q, max_values_matrix::Array{Real, 2}=max_values_matrix, a::Real = 0, h_H::Real = 0, asset_grid::LinSpace = asset_grid, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if l < 0 || c < 0 || h_H + l > 1 || h_H + l < 0
        v = -10000
    else
        v = util(c, l) + beta*dot(Q, get_value_for_middle_with_uncertainty(a_new, asset_grid, max_values_matrix))
    end
return v, a_new, l, a, w, h_H, y, c
end

value_for_young (generic function with 13 methods)

In [589]:
# calculating value for young
function value_for_young(a_new::Real, l::Real,  a::Real, w::Real, h_H::Real, v_new, Q = Q, asset_grid::LinSpace = asset_grid, tau::Real = tau, beta::Real = beta)
    y = w*(1 - tau)*(1 - h_H - l)
    #c = -100
    c = (1 + r)*a + y - a_new
    if l < 0 || c < 0 || h_H + l > 1 || h_H + l < 0
        v = -100
    else
        v = util(c, l) + beta*dot(v_new, Q)
    end
return  v, a_new, l, a, w, h_H, y, c
end

value_for_young (generic function with 13 methods)

In [590]:
# caluculate value for young 


value_matrix = zeros(num_grid^2, num_grid^3);
c_mat = zeros(num_grid^2, num_grid^3);
l_mat = zeros(num_grid^2, num_grid^3);
y_mat = zeros(num_grid^2, num_grid^3);
a_new_mat = ones(num_grid^2, num_grid^3);
w_mat = zeros(num_grid^2, num_grid^3);
a_mat = zeros(num_grid^2, num_grid^3);
h_H_mat = zeros(num_grid^2, num_grid^3);
for i = 1:num_grid
    for j = 1:num_grid
        for k = 1:num_grid
            for m = 1:num_grid
                for n = 1:num_grid
                    v_new = get_value_for_middle_with_uncertainty(asset_grid[i])# value for middle with uncertainty
                    result = value_for_young(0, l_grid[j], 0, wage_grid[m], 0, v_new)
                    value_matrix[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[1]
                    a_new_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[2]
                    l_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[3]
                    a_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[4]
                    w_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[5]
                    h_H_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[6]
                    y_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2] = result[7]
                    c_mat[i + (j - 1)num_grid, k + (m - 1)num_grid + (n - 1)num_grid^2]  = result[8]
                end
            end
        end
    end
end
# get max value for each asset, wage, h_H. maximizing with respect to new asset and leisure
max_value, max_ind = findmax(value_matrix, 1)
# reshape value matrix to asset_grid x (w x h_H)
# this should be fix throughout the imulation
max_values_reshaped = reshape(max_value, num_grid, num_grid^2);

In [591]:
# get policy function for a_new, h_M (Labor supply), c, 
# policy function for new asset
a_new_pol_young = a_new_mat[max_ind]
# policy function for labor supply
l_pol_young = l_mat[max_ind]
h_M_pol_young = 1 - l_pol_young - reshape(h_H_mat[1, :], 1, num_grid^3)
c_pol_young = c_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]
# policy function for new asset
#a_new_pol = a_new_mat[max_ind]

1×729 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  5.625  5.625  5.625  5.625  5.625

In [592]:
h_M_pol_young

1×729 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.625  0.625  0.625  0.625  0.625

In [593]:
max_values_reshaped

9×81 Array{Float64,2}:
 2.36778  3.26708  3.58103  3.81329  …  4.17022  4.31792  4.45221  4.57599
 2.36778  3.26708  3.58103  3.81329     4.17022  4.31792  4.45221  4.57599
 2.36778  3.26708  3.58103  3.81329     4.17022  4.31792  4.45221  4.57599
 2.36778  3.26708  3.58103  3.81329     4.17022  4.31792  4.45221  4.57599
 2.36778  3.26708  3.58103  3.81329     4.17022  4.31792  4.45221  4.57599
 2.36778  3.26708  3.58103  3.81329  …  4.17022  4.31792  4.45221  4.57599
 2.36778  3.26708  3.58103  3.81329     4.17022  4.31792  4.45221  4.57599
 2.36778  3.26708  3.58103  3.81329     4.17022  4.31792  4.45221  4.57599
 2.36778  3.26708  3.58103  3.81329     4.17022  4.31792  4.45221  4.57599

In [594]:
?findmax


search: findmax findmax! findmin findmin!



```
findmax(itr) -> (x, index)
```

Returns the maximum element and its index. The collection must not be empty.

```jldoctest
julia> findmax([8,0.1,-9,pi])
(8.0,1)
```

```
findmax(A, region) -> (maxval, index)
```

For an array input, returns the value and index of the maximum over the given region.


In [595]:
# simulation 100 thousand H/H
# initializing matrices
num_period = 3;
num_hh = 100 # 100000
c_record = zeros(num_hh, num_period);
a_record = zeros(num_hh, num_period);
w_record = zeros(num_hh, num_period);
h_H_record = zeros(num_hh, num_period);
h_M_record = zeros(num_hh, num_period);



In [596]:
# generate wage, care needs streams for each household

w_record = rand(w_dist, num_hh, num_period);
w_record[:,3] = 0;
h_H_record = rand(h_H_dist, num_hh, num_period);
h_H_record[:, 1] = 0;
h_H_record[:, 3] = 0;


In [597]:
# return (c, a_new, h_M) for young hh
# prepare for interpolation
c_pol_young_reshape = reshape(c_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
a_new_pol_young_reshape = reshape(a_new_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
h_M_pol_young_reshape = reshape(h_M_pol_young, num_grid, num_grid, num_grid) # reshape for interpolate    
c_itp = interpolate((asset_grid, wage_grid, h_H_grid), c_pol_young_reshape, Gridded(Linear()));
a_itp = interpolate((asset_grid, wage_grid, h_H_grid), a_new_pol_young_reshape, Gridded(Linear()));
h_M_itp = interpolate((asset_grid, wage_grid, h_H_grid), h_M_pol_young_reshape, Gridded(Linear()));

for n = 1:num_hh
    c_record[n,1] = c_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
    a_record[n,2] = a_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
    h_M_record[n,1] = h_M_itp[a_record[n,1], w_record[n,1], h_H_record[n,1]]
end




In [598]:
# return (c, a_new, h_M) for middle

c_pol_middle_reshape = reshape(c_pol_middle, num_grid, num_grid, num_grid);
a_new_pol_middle_reshape = reshape(a_new_pol_middle, num_grid, num_grid, num_grid);
h_M_pol_middle_reshape = reshape(h_M_pol_middle, num_grid, num_grid, num_grid);
c_itp = interpolate((asset_grid, wage_grid, h_H_grid), c_pol_middle_reshape, Gridded(Linear()));
a_itp = interpolate((asset_grid, wage_grid, h_H_grid), a_new_pol_middle_reshape, Gridded(Linear()));
h_M_itp = interpolate((asset_grid, wage_grid, h_H_grid), h_M_pol_middle_reshape, Gridded(Linear()));


for n = 1:num_hh
    c_record[n,2] = c_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
    a_record[n,3] = a_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
    h_M_record[n,2] = h_M_itp[a_record[n,2], w_record[n,2], h_H_record[n,2]]
end

for n = 1:num_hh
    c_record[n,3] = (1+r)*a_record[n, 3] + (w_record[n,1] + w_record[n, 2])*0.5*0.5 # replacement rate for pension is 0.5 
end

In [599]:
c_record

100×3 Array{Float64,2}:
 0.499123  0.515046  0.464705
 1.28169   1.39083   1.20442 
 1.05209   0.727192  0.801553
 0.51703   1.04164   0.890268
 0.896838  0.316362  0.543645
 0.836235  0.174625  0.457504
 0.907789  0.767412  0.790967
 0.416701  0.457277  0.410202
 0.995137  0.635133  0.739028
 0.853836  0.473858  0.595721
 0.538451  0.329343  0.392713
 2.82027   1.5402    1.93891 
 0.697363  0.408872  0.516512
 ⋮                           
 1.59576   1.08795   1.20066 
 0.630043  1.00318   0.823688
 0.4952    0.839168  0.686293
 1.12122   0.839809  0.878039
 0.403212  0.678699  0.482236
 0.643756  1.11395   0.782033
 0.601528  0.791082  0.681258
 0.372484  0.828227  0.54156 
 0.369379  1.08745   0.6491  
 0.881711  1.27309   0.979865
 0.647552  1.12369   0.80417 
 1.38034   0.830488  0.985884

In [600]:
w_record

100×3 Array{Float64,2}:
 0.887329  0.97149   0.0
 2.27856   2.53913   0.0
 1.87038   1.33583   0.0
 0.919164  2.64191   0.0
 1.59438   0.580201  0.0
 1.48664   0.343374  0.0
 1.61385   1.55002   0.0
 0.740802  0.900007  0.0
 1.76913   1.18698   0.0
 1.51793   0.864953  0.0
 0.957246  0.613604  0.0
 5.01381   2.74185   0.0
 1.23976   0.826292  0.0
 ⋮                      
 2.8369    1.96575   0.0
 1.12008   2.17468   0.0
 0.880356  1.86482   0.0
 1.99327   1.51888   0.0
 0.716822  1.21212   0.0
 1.14445   1.98368   0.0
 1.06938   1.65565   0.0
 0.662195  1.50404   0.0
 0.656674  1.93973   0.0
 1.56749   2.35197   0.0
 1.1512    2.06548   0.0
 2.45394   1.48959   0.0

In [601]:
h_M_record

100×3 Array{Float64,2}:
 0.733801  0.67262   0.0
 0.625     0.608618  0.0
 0.625     0.604862  0.0
 0.724251  0.438084  0.0
 0.625     0.806787  0.0
 0.625     0.837051  0.0
 0.625     0.550108  0.0
 0.777759  0.669533  0.0
 0.625     0.613444  0.0
 0.625     0.724229  0.0
 0.712826  0.787291  0.0
 0.625     0.624153  0.0
 0.628073  0.67692   0.0
 ⋮                      
 0.625     0.614948  0.0
 0.663977  0.512554  0.0
 0.735893  0.5       0.0
 0.625     0.614346  0.0
 0.784953  0.633502  0.0
 0.656664  0.623951  0.0
 0.679185  0.530897  0.0
 0.801342  0.611852  0.0
 0.802998  0.622912  0.0
 0.625     0.601429  0.0
 0.654639  0.604483  0.0
 0.625     0.619475  0.0

In [602]:
h_H_record

100×3 Array{Float64,2}:
 0.0  0.0359329    0.0
 0.0  0.0163821    0.0
 0.0  0.0201382    0.0
 0.0  0.311916     0.0
 0.0  0.0191532    0.0
 0.0  0.059937     0.0
 0.0  0.0748919    0.0
 0.0  0.0604648    0.0
 0.0  0.0304625    0.0
 0.0  0.0162855    0.0
 0.0  0.0286274    0.0
 0.0  0.000847457  0.0
 0.0  0.0751923    0.0
 ⋮                    
 0.0  0.0100525    0.0
 0.0  0.112446     0.0
 0.0  0.12608      0.0
 0.0  0.0106536    0.0
 0.0  0.00286077   0.0
 0.0  0.00104934   0.0
 0.0  0.0941026    0.0
 0.0  0.0131484    0.0
 0.0  0.00208843   0.0
 0.0  0.0235708    0.0
 0.0  0.0205169    0.0
 0.0  0.0055248    0.0

In [603]:
sum(h_M_record)

126.61354078012988

In [604]:
sum(c_record)

304.99967444168936

LoadError: UndefVarError: plotly not defined